In [11]:
import torchvision
import torch.nn as nn
import torch

In [12]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
model.roi_heads.box_predictor.cls_score = nn.Linear(1024, 1, bias=True)
model.roi_heads.box_predictor.bbox_pred = nn.Linear(1024, 4, bias=True)
# model.load_state_dict(torch.load('model.pth'))

In [13]:
#Opcjonalne zamrożenie wag
weights_dict = dict(model.named_parameters())
for k, v in weights_dict.items():
    if "box_predictor" not in k:
        v.requires_grad = False

In [14]:
#model.roi_heads.box_predictor

In [15]:
# images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 1, 4)
# boxes[:, :, 2:4] = boxes[:, :, 0:2] + boxes[:, :, 2:4]
# labels = torch.zeros((4, 11), dtype=torch.int64)
# images = list(image for image in images)
# targets = []
# for i in range(len(images)):
#     d = {}
#     d['boxes'] = boxes[i]
#     d['labels'] = labels[i]
#     targets.append(d)
# output = model(images, targets)

In [16]:
import json
import torch
from PIL import Image
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


# Wczytaj plik JSON
with open('labelj.json') as f:
    data = json.load(f)


In [17]:
# import torchvision.transforms as transforms  # Transformations we can perform on our dataset
# my_transforms = transforms.Compose(
#     [  # Compose makes it possible to have many transforms
#         transforms.RandomCrop((32, 32)),  # Takes a random (32,32) crop
#         transforms.ColorJitter(brightness=0.5),  # Change brightness of image
#         transforms.RandomRotation(
#             degrees=45
#         ),  # Perhaps a random rotation from -45 to 45 degrees
#         transforms.RandomHorizontalFlip(
#             p=0.5
#         ),  # Flips the image horizontally with probability 0.5
#         transforms.RandomVerticalFlip(
#             p=0.05
#         ),  # Flips image vertically with probability 0.05
#         transforms.RandomGrayscale(p=0.2),  # Converts to grayscale with probability 0.2
#         transforms.ToTensor(),  # Finally converts PIL image to tensor so we can train w. pytorch
#         transforms.Normalize(
#             mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]
#         ),  # Note: these values aren't optimal
#     ]
# )

In [18]:
##################################################NOWY
import torchvision.transforms as T
class TrainingDataset(torch.utils.data.Dataset):
    def __init__(self, images, targets):
        self.images = []
        self.boxes = []
        

        for item in data:

            file_name = item['data']['image'].split('-')[-1]
    
            image_path = r'C:\Users\Boows\Desktop\Pulpit\PWR\NEURONOWKA\sieci\\' + file_name  # Zmień odpowiednio ścieżkę do katalogu ze zdjęciami
            image = Image.open(image_path)
    
            transform = Resize((600, 1200))
            to_tensor = ToTensor()
            image = transform(image)
            image = to_tensor(image)

            self.images.append(image)

            
            points = item['annotations'][0]['result'][0]['value']['points']
    
            xmin = min(point[0] for point in points)
            ymin = min(point[1] for point in points)
            xmax = max(point[0] for point in points)
            ymax = max(point[1] for point in points)
            box = torch.tensor([[xmin, ymin, xmax, ymax]])
            self.boxes.append(box)
            self.boxes = targets
        self.labels = torch.zeros((len(images), 1), dtype=torch.int64)


        self.transforms = T.Compose([
        T.RandomApply(T.CenterCrop(10),p=0.5),
        T.RandomApply(T.PILToTensor(),p=0.5),
        T.RandomApply(T.ElasticTransform(alpha=45.0),p=0.5),
        T.RandomApply(T.ConvertImageDtype(torch.float),p=0.5)])

    # def crop_img_with_box(self, img, box):
    #     # TODO: crop image with box
    #     return img, box

    def __getitem__(self, idx):
        out_img = self.images[idx]
        if self.transforms is not None:
            out_img = self.transforms(out_img)

        out_box = self.boxes[idx]
        out_img, out_box = self.crop_img_with_box(out_img, out_box)

        return out_img, out_box, self.labels[idx]

    def __len__(self):
        return len(self.images)



In [19]:
#dataset = TrainingDataset(images, transforms=my_transforms)
# Pobierz pierwszy obraz z listy

# images[12].shape
# boxes[0]
data



[{'id': 1,
  'annotations': [{'id': 1,
    'completed_by': 1,
    'result': [{'original_width': 1536,
      'original_height': 2048,
      'image_rotation': 0,
      'value': {'points': [[14.192487384209254, 8.59448138433522],
        [88.99872297181221, 9.037996615091759],
        [92.30438982505096, 94.3938499517012],
        [8.228685914636326, 92.91805302135883]],
       'closed': True,
       'polygonlabels': ['kartka']},
      'id': 'uWscj3vOH4',
      'from_name': 'label',
      'to_name': 'image',
      'type': 'polygonlabels',
      'origin': 'manual'}],
    'was_cancelled': False,
    'ground_truth': False,
    'created_at': '2023-05-10T23:11:05.421553Z',
    'updated_at': '2023-05-10T23:11:05.421589Z',
    'lead_time': 106.427,
    'prediction': {},
    'result_count': 0,
    'unique_id': '287b96a9-d6e7-4da0-bc48-b6b5f7808900',
    'last_action': None,
    'task': 1,
    'project': 1,
    'updated_by': 1,
    'parent_prediction': None,
    'parent_annotation': None,
    'las

In [20]:
from tqdm.notebook import tqdm

#Hiperparametr
epochs = 10
#lr, momentum - hiperparametry
#można spróbować torch.optim.Adam 
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
training_dataset = TrainingDataset(data, data)
#batch_size = 2 - hiperparametr
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=2, shuffle=True)
val_dataset = TrainingDataset(data,data)
val_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=2, shuffle=True)

for epoch in tqdm(list(range(epochs))):
    for images, boxes, labels in training_dataloader:
        images = list(image.to(device) for image in images)
        targets = []
        for i in range(len(images)):
            d = {}
            d['boxes'] = boxes[i].to(device)
            d['labels'] = labels[i].to(device)
            targets.append(d)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(losses)

    for images, boxes, labels in val_dataloader:
        images = list(image.to(device) for image in images)
        targets = []
        for i in range(len(images)):
            d = {}
            d['boxes'] = boxes[i].to(device)
            d['labels'] = labels[i].to(device)
            targets.append(d)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        print(f"Val loss: {losses.item()}")

#Zapisanie modelu
torch.save(model.state_dict(), "model.pth")

IndexError: too many indices for tensor of dimension 2